In [2]:
import socket
ip_dict = {"1A": "192.168.1.151", "1B": "192.168.1.151", "1A'": "192.168.1.151", "1B'": "192.168.1.151", 
           "2A": "192.168.1.152", "2B": "192.168.1.152", "2A'": "192.168.1.152", "2B'": "192.168.1.152", 
           "1C": "192.168.1.153", "2C": "192.168.1.153"}
motor_dict = {"1A": "1", "1B": "2", "1A'": "3", "1B'": "4", "2A": "1", "2B": "2", "2A'": "3", "2B'": "4", "1C": "1", "2C": "2"}

def sendinstr(instr,motor,respond = False):
    clientsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        clientsock.connect((ip_dict[motor],23))
        clientsock.recv(4096) # The picomotor will return a dummy code when connected successfully
    except ConnectionRefusedError as msg:
        raise
    clientsock.sendall((instr+'\r').encode())
    if respond:
        response = clientsock.recv(4096).decode()
        print("Response of "+instr+": "+response)
    clientsock.close()
    if respond:
        return response

def set_speed(motor,value):
    # Value is in step/sec
    sendinstr("%sVA%u\rSM\r" %(motor_dict[motor],value),motor)

def set_accel(motor,value):
    # Value is in step/sec^2
    sendinstr("%sAC%u\rSM\r" %(motor_dict[motor],value),motor)

def get_speed(motor):
    # Returned value is in step/sec
    return sendinstr("%sVA?" %(motor_dict[motor]),motor,True)

def get_accel(motor):
    # Returned value is in step/sec
    return sendinstr("%sAC?" %(motor_dict[motor]),motor,True)

def set_zero(motor,value=0):
    # Set current position as the assign value (by default '0')
    sendinstr("%sDH%u\rSM\r" %(motor_dict[motor],value),motor)

def get_position(motor):
    # Returned value is the current position respect to '0'
    return sendinstr("%sTP?" %(motor_dict[motor]),motor,True)

def check_error(motor):
    # Returned all errors on a driver
    return sendinstr("TB?",motor,True)

def get_all(motor):
    # Returned all settings above
    print("Motor: ",motor)
    print("Current Position: ",get_position(motor))
    print("Current Speed: ",get_speed(motor))
    print("Current Acceleration: ",get_accel(motor))